In [75]:
# импорт необходимых библиотек
import random
import numpy as np
import scipy.stats as st
from statsmodels.stats.power import tt_ind_solve_power, zt_ind_solve_power
from statsmodels.stats.proportion import proportion_effectsize
from statsmodels.stats.meta_analysis import effectsize_smd
from statsmodels.stats import proportion
from typing import Union
from math import ceil
from tqdm import tqdm

## Задача:
На основе выбранных продуктов из первого вебинара, выберите 2 A/B-теста и составьте план эксперимента, чтобы избежать Peeking Problem \
Распишите, какие метрики и как вы будете анализировать

## План эксперимента для первого А/Б-теста

### Составление бэклога экспериментов и приоритизация осуществлялась на основании ICE методики. Первой на основании ICE score выбрана следующая гипотеза:

### 1.	Гипотеза: 
Использование алгоритмов автоматического выбора аудитории, чтобы система на сайте показывала рекламу тем, кто с большей вероятностью совершит конверсию, для всех групп покупателей увеличит валовую выручку на 10%, потому что увеличится частота покупок.

### 2.	Что делаем: 
Разрабатываем две версии сайта. Первая (версия А - экспериментальная) - с алгоритмом создания профиля авторизированного покупателя и алгоритмом позиционирования рекламных объявлений в зависимости от количественных параметров этого профиля (прежде всего параметра – байесовской вероятности совершения покупки при условии просмотра рекламы). Вторая (версия Б - контрольная) – ныне существующая версия сайта без указанного алгоритма.

### 3.	На каких пользователях тестируем: 
Hа всех поло-возрастных группах пользователей. Для эксперимента выделяем на основе рандомизированного выбора 10% от аудитории сайта, разделяя эти 10% случайным образом на две равные выборки – в контрольную и экспериментальную группы.

### 4.	Ключевые метрики для оценки эксперимента: 
Валовая выручка (основная метрика) и конверсия / коэффициент конверсии (дополнительная метрика) в контрольной и экспериментальной выборках.

### 5.	Ожидаемый эффект: 
Cтатистически значимое различие (p-value < 0,05, α = 0,95) в валовой выручке и конверсии, превосходящее 10% в экспериментальной группе относительно контрольной. 

In [47]:
### Расчет длительности эксперимента для метрики "валовая выручка" при заданных условиях:

### Инициализация исходных параметров

mean_control = 100_000 ### допустим, у нас в средней такая валовая выручка в контрольной группе
mean_test = 100_000 * 1.1
std_control, std_test = 90_000, 90_000 * 1.15 #### учитываем возможное увеличение SD 
n_control , n_test = 10_000, 10_000
alpha = 0.05
beta = 0.2

In [48]:
#Расчёт effect_size для непрерывных метрик встроенной функцией

es_normalized = effectsize_smd(mean_control, std_control, n_control, mean_test, std_test, n_test)[0]
es_normalized

-0.10310467054893516

In [49]:
# Функция для расчёта минимально необходимой выборки * 2(для теста и контроля) для непрерывной метрики
def calc_sample_size_continuous(effect_size: float,
                                alpha: float = .05,
                                beta: float = .2,
                                ratio: Union[float, int] = 1):
    
    n = tt_ind_solve_power(effect_size=effect_size,
                           alpha=alpha,
                           power=(1 - beta),
                           ratio=ratio,
                  )
    return int(n * 2)

In [50]:
# Расчёт минимально необходимой выборки * 2(для теста и контроля) для непрерывной метрики

sample_size = calc_sample_size_continuous(effect_size = es_normalized, alpha = alpha, beta = beta)
sample_size

2955

In [51]:
### Допустим наш магазин посещает 1000 человек в день, таким образом 10% от аудитории, 
### выделенных для эксперимента, потребует 100 человек в день

n_days = ceil(sample_size / 100) ### количество дней округляем до целого в большую сторону, чтобы размеры выборок гарантированно дали желаемый результат
n_days

### Вывод: на проведение эксперимента нужно 30 дней.

30

In [66]:
# Проведем моделирование AB-теста
alpha = .05
sample_A_mean, sample_B_mean = 100_000, 100_000 * 1.1 
sample_A_size, sample_B_size = 1500, 1500 ### предположим нам удалось получить точное разделение выборок на контрольную и тестовую
sample_A_se, sample_B_se = std_control/(sample_A_size ** 0.5), std_test/(sample_B_size ** 0.5)

# Генерируем нормальное распределение на основе введенных данных
sample_A = st.norm(loc=sample_A_mean, scale=sample_A_se).rvs(size=sample_A_size, random_state=42)
sample_B = st.norm(loc=sample_B_mean, scale=sample_B_se).rvs(size=sample_B_size, random_state=42)

In [67]:
# Проведем АБ-тестирование с помощью бутсрапа
booted_diff = []
booted_pvalue = []
size = max(sample_A.size, sample_B.size)
for _ in tqdm(range(10_000)):
    a_s = np.random.choice(sample_A, size=size, replace=True)
    b_s = np.random.choice(sample_B, size=size, replace=True)
    booted_diff.append(np.mean(a_s - b_s))
    booted_pvalue.append(st.ttest_ind(a_s, b_s, equal_var=False, alternative='two-sided')[1])

# Доверительный интервал разниц между группами
md_ci, std_ci = np.mean(booted_diff), np.std(booted_diff, ddof=1)
left_ci, right_ci = np.percentile(booted_diff, [2.5, 97.5])
p_value_ci = 2 * (1 - st.norm.cdf(np.abs(md_ci / std_ci)))

100%|██████████| 10000/10000 [00:02<00:00, 3669.60it/s]


In [68]:
# Доверительный интервал разниц между группами: контрольная группа (A) будет под номером 1, а
# экспериментальная группа (В) будет под номером 2
print(f"Средняя разница: {np.round(md_ci, 5)}, p-value: {p_value_ci} и доверительный интервал: [{np.round(left_ci, 5)}, {np.round(right_ci, 5)}]")

if p_value_ci < alpha and not min(left_ci, right_ci) < 0 < max(left_ci, right_ci):
    print(f"Группа # {1 if md_ci > 0 else 2} имеет большее среднее")
else:
    print("Нет статистически значимой разницы")

Средняя разница: -10018.51978, p-value: 0.0 и доверительный интервал: [-10188.87561, -9842.00073]
Группа # 2 имеет большее среднее


### 6.	План действий в зависимости от результатов эксперимента: 
Если статистически значимое различие метрик наблюдается, то масштабируем алгоритм на весь сайт для 100% пользователей. Если значимого различия не получено, то переходим к разработке дизайна эксперимента для следующей по величине ICE score гипотезы.
#### Вывод: статистически значимое различие получено - алгоритм можно внедрять на сайт.

## План эксперимента для второго А/Б-теста

### Составление бэклога экспериментов и приоритизация осуществлялась на основании ICE методики.  Вторым АБ-тестом является тест для проверки гипотезы, набравшей следующее по величине значение ICE score, а именно:

### 1.	Гипотеза: 
Подсветка и приоритетный вывод на первую страницу (или в первый горизонтальный слайдер) товаров и акций, наиболее характерных для половозрастной группы пользователя (например: мобильную и компьютерную технику для молодых мужчин, галантерею и аксессуары для мобильных устройств для молодых женщин, книги и бытовую технику для женщин среднего возраста, товары для сада/огорода для пожилых покупателей), для всех групп пользователей позволит увеличить конверсию на 10% и средний чек на 20%, потому что увеличится предложение клиентам интересующих их товаров.

### 2.	Что делаем: 
Разрабатываем две версии сайта. Первая (версия А - экспериментальная) - с алгоритмом вывода целевых для авторизированного пользователя товаров и акций в зависимости от групповой принадлежности пользователя.  Вторая (версия Б - контрольная) – ныне существующая версия сайта без указанного алгоритма.

### 3.	На каких пользователях тестируем: 
Hа первом этапе можно выбрать в качетстве целевой группы молодых мужщин в возрасте от 25 до 35 лет. Для эксперимента выделяем на основе рандомизированного выбора 10% от аудитории сайта из целевой группы, разделяя эти 10% случайным образом на две равные выборки – в контрольную и экспериментальную группы.

### 4.	Ключевые метрики для оценки эксперимента: 
Конверсия (основная метрика) и средний чек (дополнительная метрика).

### 5.	Ожидаемый эффект: 
Cтатистически значимое различие (p-value < 0,05, α = 0,95) в конверсии, превосходящее или равное 10% в экспериментальной группе относительно контрольной. 

In [69]:
### Расчет длительности эксперимента для метрики "конверсия" при заданных условиях:

### Инициализация исходных параметров - допустим исходная конверсия у нас 40%

conv_sample_A, conv_sample_B = 0.4, 0.4*1.1  
alpha = 0.05
beta = 0.2

In [70]:
# Расчёт effect_size для пропорций

es_prop = proportion_effectsize(conv_sample_A, conv_sample_B)
es_prop

-0.08106803839554289

In [71]:
# Формула для расчёта минимально необходимой выборки * 2 (для теста и контроля) для пропорций
def calc_sample_size_proportion(effect_size: float,
                                alpha: float = .05,
                                beta: float = .2,
                                ratio: Union[float, int] = 1):
    
    n = zt_ind_solve_power(effect_size=effect_size,
                           alpha=alpha,
                           power=(1 - beta),
                           ratio=ratio,
                  )
    return int(n * 2)

In [72]:
# Расчёт минимально необходимой выборки * 2 (для теста и контроля) для пропорций

sample_size_prop = calc_sample_size_proportion(effect_size = es_prop, alpha = alpha, beta = beta)
sample_size_prop

4777

In [73]:
### Допустим наш магазин посещает 1000 человек в день, таким образом 10% от аудитории, 
### выделенных для эксперимента, потребует 100 человек в день (из них 50 человек будет из нашей 
### целевой аудитории).

n_days_prop = ceil(sample_size / 50) ### количество дней округляем до целого в большую сторону, чтобы размеры выборок гарантированно дали желаемый результат
n_days_prop

### Вывод: на проведение эксперимента нужно 60 дней.

60

In [74]:
# Проведем моделирование AB-теста

# Задаем начальные параметры
alpha = .05
sample_1_size, sample_2_size = 5000, 5000 ## допустим нам удалось собрать выборки нужного размера
prob_1, prob_2 = 2000 / sample_1_size, 2200 / sample_2_size
# Генерируем распределение
a = st.bernoulli.rvs(p=prob_1, size=sample_1_size, random_state=12)
b = st.bernoulli.rvs(p=prob_2, size=sample_2_size, random_state=25)

In [76]:
# Сравнение хи2 и z
chi_value, p_value_chi2, table = proportion.proportions_chisquare([a.sum(), b.sum()], [a.size, b.size])
z_value, p_value_z = proportion.proportions_ztest([a.sum(), b.sum()], [a.size, b.size])
print(p_value_chi2, p_value_z)

#### Вывод: наблюдаются статистически значимые различия между контрольной и тестовой группами

4.997862747090972e-07 4.997862747091e-07


In [81]:
#### усиливаем расчеты бутстрепом

booted_diff = []
booted_pvalue = []
size = max(a.size, b.size)
for _ in tqdm(range(10_000)):
    a_s = st.bernoulli.rvs(p=prob_1, size=size)
    b_s = st.bernoulli.rvs(p=prob_2, size=size)
    booted_diff.append(np.mean(a_s - b_s))
    booted_pvalue.append(proportion.proportions_chisquare([a_s.sum(), b_s.sum()], [a_s.size, b_s.size])[1])

md_ci, std_ci = np.mean(booted_diff), np.std(booted_diff, ddof=1)
left_ci, right_ci = np.percentile(booted_diff, [2.5, 97.5])
p_value_ci = 2 * (1 - st.norm.cdf(np.abs(md_ci / std_ci)))

100%|██████████| 10000/10000 [00:06<00:00, 1648.94it/s]


In [83]:
# Доверительный интервал разниц между группами

print(f"Средняя разница: {np.round(md_ci, 5)}, p-value: {p_value_ci} и доверительный интервал: [{np.round(left_ci, 5)}, {np.round(right_ci, 5)}]")

if p_value_ci < alpha and not min(left_ci, right_ci) < 0 < max(left_ci, right_ci):
    print(f"Группа # {1 if md_ci > 0 else 2} имеет большее среднее.")
else:
    print("Нет статистически значимой разницы")

Средняя разница: -0.04003, p-value: 4.6514995655222435e-05 и доверительный интервал: [-0.0596, -0.0206]
Группа # 2 имеет большее среднее.


### 6.	План действий в зависимости от результатов эксперимента: 

#### Вывод: статистически значимое увеличение конверсии получено, но среднее увеличение конверсии наблюдается в размере 4%, а не 10% (как планировалось изначально). Поэтому решение о внедрении предлагаемых изменений на сайт должно приниматься руководством исходя их этого результата.